In [1]:
from decouple import config

In [3]:
LOCALHOST_PW = config('LOCALHOST_PW')


UndefinedValueError: LOCALHOST_PW not found. Declare it as envvar or define a default value.

# Playback Sawthooth Waveform

In [12]:
# AOM Switch 
import pylabnet.hardware.ni_daqs.nidaqmx_card as nidaqmx
import pylabnet.network.client_server.nidaqmx_card as nidaqmx_card_server
import numpy as np
import time

In [13]:
aom_daq = nidaqmx_card_server.Client(
    host='140.247.189.82',
    port=24195
)

In [28]:
amplitude = 1 # in V
offset = 0 # in V
num_points = 50
frequency = 0.02 # in Hz

In [29]:
period = 1/frequency
scanning_range_up = np.linspace(offset, offset+amplitude, num_points)
scanning_range_down = scanning_range_up[::-1]

In [30]:

for voltage in np.concatenate((scanning_range_up,scanning_range_down)):
    aom_daq.set_ao_voltage('ao4', voltage)
    time.sleep(period / (2 * num_points))


In [27]:
aom_daq.set_ao_voltage('ao4', 0)
